In [81]:
# import sys
# print(sys.executable)

In [82]:
# !{sys.executable} -m pip install rasterstats

In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geopandas as gpd
import rasterio
import rasterio.mask
import requests, zipfile, io, os
from tqdm import tqdm
import random
from rasterstats import zonal_stats

#### First we take all countries of Africa by code below, if we want only one country we download from GADM or Natural Earth. 

In [84]:
import geopandas as gpd
import requests, zipfile, io, os
from tqdm import tqdm

# 🌍 List of African countries (ISO codes for GADM)
africa_countries = [
    'DZA', 'AGO', 'BEN', 'BWA', 'BFA', 'BDI', 'CMR', 'CPV', 'CAF', 'TCD',
    'COM', 'COG', 'COD', 'DJI', 'EGY', 'GNQ', 'ERI', 'SWZ', 'ETH', 'GAB',
    'GMB', 'GHA', 'GIN', 'GNB', 'CIV', 'KEN', 'LSO', 'LBR', 'LBY', 'MDG',
    'MWI', 'MLI', 'MRT', 'MUS', 'MAR', 'MOZ', 'NAM', 'NER', 'NGA', 'RWA',
    'STP', 'SEN', 'SYC', 'SLE', 'SOM', 'ZAF', 'SSD', 'SDN', 'TZA', 'TGO',
    'TUN', 'UGA', 'ZMB', 'ZWE'
]

# 🗂 Create output folder
os.makedirs("africa_shapefiles", exist_ok=True)

# 🧭 Function to download and extract shapefile
def download_gadm(iso):
    url = f"https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_{iso}_shp.zip"
    try:
        r = requests.get(url, stream=True)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(f"africa_shapefiles/{iso}")
        return gpd.read_file(f"africa_shapefiles/{iso}/gadm41_{iso}_1.shp")
    except Exception as e:
        print(f"❌ Failed for {iso}: {e}")
        return None

# 🗺️ Download and merge all provinces
provinces = []
for iso in tqdm(africa_countries, desc="Downloading countries"):
    shp = download_gadm(iso)
    if shp is not None:
        shp["country"] = iso
        provinces.append(shp)

# 🧩 Merge all into one shapefile
africa_provinces = gpd.GeoDataFrame(pd.concat(provinces, ignore_index=True))
africa_provinces.to_file("africa_provinces.shp")

print("✅ All African provinces merged and saved as 'africa_provinces.shp'")


✅ All African provinces merged and saved as 'africa_provinces.shp'


C:\ProgramData\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'country' to 'country_1'
  ogr_write(


In [85]:
africa_provinces.head(2)

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry,country
0,DZA.1_1,DZA,Algeria,Adrar,Duperré,ولاية أدرا,Wilaya,Province,1,DZ.AR,DZ-01,"POLYGON ((1.42512 23.37227, 1.42495 23.36762, ...",DZA
1,DZA.2_1,DZA,Algeria,Aïn Defla,Ain Dafla,ولاية عين الدفلى,Wilaya,Province,44,DZ.AD,NA,"MULTIPOLYGON (((2.52087 36.22804, 2.51443 36.2...",DZA


#### Let's obtain calculate NDVI and NDWI per province.

In [86]:
# --------------------------
# 1. Load provinces shapefile
# --------------------------
provinces = gpd.read_file("africa_provinces.shp")

# --------------------------
# 2. Load Sentinel-2 image
# --------------------------
# Example band indices — adjust to your file band order!
# B8 = NIR, B4 = RED, B3 = GREEN

for i in range(20,25):
    with rasterio.open(f"Sentinel_{i}.tif") as src:
        nir = src.read(8).astype('float32')  # band 8 = NIR
        red = src.read(4).astype('float32')  # band 4 = Red
        green = src.read(3).astype('float32')  # band 3 = Green
        profile = src.profile

# --------------------------
# 3. Compute NDVI & NDWI
# --------------------------
ndvi = (nir - red) / (nir + red + 1e-6)
ndwi = (green - nir) / (green + nir + 1e-6)

# --------------------------
# 4. Function to get mean values per province
# --------------------------

for i in range(20, 25):
    def zonal_stats(image, shapes, src_profile):
        results = []
        for _, row in shapes.iterrows():
            geom = [row['geometry']]
            try:
                with rasterio.open(f"Sentinel_{i}.tif") as src:
                    out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
                    out_image = out_image.astype('float32')
                    # Extract bands again for local NDVI/NDWI
                    nir = out_image[7]
                    red = out_image[3]
                    green = out_image[2]
                    ndvi = (nir - red) / (nir + red + 1e-6)
                    ndwi = (green - nir) / (green + nir + 1e-6)
                    results.append({
                        'province': row['NAME_1'],  # column name may differ
                        'mean_NDVI': np.nanmean(ndvi),
                        'mean_NDWI': np.nanmean(ndwi)
                    })
            except Exception as e:
                print(f"Error on {row['NAME_1']}: {e}")
        return pd.DataFrame(results)

# --------------------------
# 5. Compute stats
# --------------------------
df = zonal_stats(ndvi, africa_provinces, profile)
print(df)

# --------------------------
# 6. Save to CSV
# --------------------------
df.to_csv("africa_provinces_ndvi_ndwi.csv", index=False)
print("✅ Saved NDVI and NDWI by province to CSV!")


Error on Agalega Islands: Input shapes do not overlap raster.
Error on Black River: Input shapes do not overlap raster.
Error on Flacq: Input shapes do not overlap raster.
Error on Grand Port: Input shapes do not overlap raster.
Error on Moka: Input shapes do not overlap raster.
Error on Pamplemousses: Input shapes do not overlap raster.
Error on Plaines Wilhems: Input shapes do not overlap raster.
Error on Port Louis: Input shapes do not overlap raster.
Error on Rivière du Rempart: Input shapes do not overlap raster.
Error on Rodriguez: Input shapes do not overlap raster.
Error on Saint Brandon: Input shapes do not overlap raster.
Error on Savanne: Input shapes do not overlap raster.
Error on Anse aux Pins: Input shapes do not overlap raster.
Error on Anse Boileau: Input shapes do not overlap raster.
Error on Anse Étoile: Input shapes do not overlap raster.
Error on Anse Royale: Input shapes do not overlap raster.
Error on Au Cap: Input shapes do not overlap raster.
Error on Baie Laza

### Note:
Perfect ✅ Sanaullah — this result means your NDVI/NDWI extraction worked successfully for all provinces that actually overlapped your Sentinel-2 image!

#### Why you still see those “Error on ... Input shapes do not overlap raster”

Those warnings are normal and expected.

They simply mean that:

The listed provinces (like Agalega Islands, Seychelles districts, etc.) are small islands located outside the area your Sentinel image covers.

Your raster didn’t include those coordinates, so rasterio.mask couldn’t clip them.

➡️ Good news:
They were automatically skipped, and your CSV was still created correctly with the valid provinces (813 total).

In [87]:
df.head(2)

,province,mean_NDVI,mean_NDWI
0,Adrar,0.034503,-0.120302
1,Aïn Defla,0.163364,-0.235962


In [88]:
df = df.drop('province', axis=1)

In [89]:
df.head(4)

,mean_NDVI,mean_NDWI
0,0.034503,-0.120302
1,0.163364,-0.235962
2,0.047870,-0.099186
3,0.113331,-0.148800


## WorldClim Files Parsing and Preporcessing

#### File = POWER_Regional_Monthly_2020_2024.csv

In [90]:
prm = pd.read_csv('Temperature')
prm.head(4)

,PARAMETER,YEAR,LAT,LON,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,T2M,2020,-0.5,21.250,25.35,25.76,25.47,25.57,25.32,24.84,24.32,24.51,24.26,24.19,24.23,24.56,24.86
1,T2M,2020,-0.5,21.875,25.06,25.46,25.17,25.34,25.04,24.60,24.14,24.25,24.04,23.98,23.99,24.27,24.61
2,T2M,2020,-0.5,22.500,24.74,25.30,25.21,25.36,25.10,24.69,24.25,24.12,24.10,23.96,23.91,24.07,24.56
3,T2M,2020,-0.5,23.125,24.65,25.68,26.03,25.86,25.97,25.58,24.74,24.11,24.38,24.22,24.00,23.92,24.92


In [91]:
Temp_df = pd.DataFrame(prm['ANN'])
Temp_df = Temp_df[['ANN']].rename(columns={'ANN': 'Temperature'})

Temp_df.head(4)

,Temperature
0,24.86
1,24.61
2,24.56
3,24.92


In [92]:
rainfall_df = pd.read_csv('Rainfall')
rainfall_df.head(4)

,PARAMETER,YEAR,LAT,LON,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,PRECTOTCORR,2020,-0.5,20.000,2.06,3.36,5.13,4.41,8.98,4.74,7.75,3.43,6.78,6.35,7.33,5.12,5.46
1,PRECTOTCORR,2020,-0.5,20.625,2.45,3.68,3.74,3.14,7.30,4.64,8.56,4.56,5.98,5.55,6.59,4.19,5.04
2,PRECTOTCORR,2020,-0.5,21.250,3.09,4.16,4.09,3.20,6.54,3.44,6.43,5.99,5.54,5.79,7.23,4.58,5.01
3,PRECTOTCORR,2020,-0.5,21.875,3.02,4.43,4.13,3.35,6.81,3.42,5.81,5.73,5.49,6.00,7.42,4.68,5.03


In [93]:
rainfall_df = pd.DataFrame(rainfall_df['ANN'])

rainfall_df = rainfall_df[['ANN']].rename(columns={'ANN': 'Rainfall'})

rainfall_df.head(4)

,Rainfall
0,5.46
1,5.04
2,5.01
3,5.03


## FAO Files Parsing and Preporcessing

### Obtain datas of Soil_OC, Soil_PH and Soil_N

#### Accurate: download SoilGrids GeoTIFF(s) and compute zonal statistics per province

Advantages: zonal average over each polygon (not just centroid), more accurate. 

Disadvantage: you must download the raster layers (GBs) or clip to your AOI.

ISRIC SoilGrids download page: https://soilgrids.org/
 or direct dataset pages (they offer GeoTIFFs by property and depth). Example filenames: ocsd0-5cm.tif etc or SoilGrids 250m layers.

In [94]:
#ls

In [95]:
import rasterio
from rasterio.merge import merge
import os

regions = ["na", "wa", "ca", "ea", "sa"]
soils = ["oc", "n", "ph"]

for s in soils:
    files = [f"{r}__soil_{s}.tif" for r in regions if os.path.exists(f"{r}__soil_{s}.tif")]
    if not files:
        print(f"⚠️ No files found for {s}")
        continue
    
    src_files = [rasterio.open(f) for f in files]
    mosaic, out_trans = merge(src_files)
    
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })
    
    out_path = f"africa_soil_{s}.tif"
    with rasterio.open(out_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    
    print(f"✅ Saved {out_path}")


✅ Saved africa_soil_oc.tif
✅ Saved africa_soil_n.tif
✅ Saved africa_soil_ph.tif


In [96]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats

# Load provinces shapefile
provinces = gpd.read_file("africa_provinces.shp").to_crs("EPSG:4326")

# Use merged continental rasters
oc_raster = "africa_soil_oc.tif"
n_raster  = "africa_soil_n.tif"
ph_raster = "africa_soil_ph.tif"

# Compute mean for each province
oc_stats = zonal_stats(provinces, oc_raster, stats=["mean"], geojson_out=False, nodata=None)
n_stats  = zonal_stats(provinces, n_raster, stats=["mean"], geojson_out=False, nodata=None)
ph_stats = zonal_stats(provinces, ph_raster, stats=["mean"], geojson_out=False, nodata=None)

# Combine into one DataFrame
soil_df = pd.DataFrame({
    "soil_OC": [r["mean"] if r["mean"] is not None else 0 for r in oc_stats],
    "soil_N": [r["mean"] if r["mean"] is not None else 0 for r in n_stats],
    "soil_pH": [r["mean"] if r["mean"] is not None else 0 for r in ph_stats]
})

soil_df.to_csv("africa_province_soil_properties.csv", index=False)
print("✅ Saved africa_province_soil_properties.csv")


✅ Saved africa_province_soil_properties.csv


In [97]:
soil_df

,soil_OC,soil_N,soil_pH
0,4.457862,8.649378,9.162534
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000
...,...,...,...
845,150.752067,112.491903,59.392828
846,141.564199,112.536212,64.609620
847,120.581660,95.294119,63.023304
848,109.520076,94.725855,66.494269


##### So this is pandas dataframe I will using mask make all 0 to np.nan , then fillna

In [98]:
soil_df=soil_df.mask(soil_df==0).fillna(soil_df.mean())
soil_df

,soil_OC,soil_N,soil_pH
0,4.457862,8.649378,9.162534
1,186.690740,142.174011,42.504298
2,186.690740,142.174011,42.504298
3,186.690740,142.174011,42.504298
4,186.690740,142.174011,42.504298
...,...,...,...
845,150.752067,112.491903,59.392828
846,141.564199,112.536212,64.609620
847,120.581660,95.294119,63.023304
848,109.520076,94.725855,66.494269


##### To calculate yield (t/ha) from FAO stats you need three elements:

1: Production (tonnes)

2: Area harvested (hectares)

3: Then:
           
           Yield (t/ha) = Production / Area harvested

In [169]:
fao_df = pd.read_csv('FAO.csv')
fao_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,12,Algeria,5312,Area harvested,221,"Almonds, in shell",2023,2023,ha,38544.0,I,Value imputed by a receiving agency,NaN
1,QCL,Crops and livestock products,12,Algeria,5412,Yield,221,"Almonds, in shell",2023,2023,kg/ha,1806.7,E,Estimated value,NaN


In [170]:
# Check available elements
print(fao_df["Element"].unique())

['Area harvested' 'Yield' 'Production']


In [171]:
# Filter rows containing 'Yield'
yield_df = fao_df[fao_df["Element"].str.contains("Yield", case=False, na=False)].copy()

# Rename for clarity
yield_df = yield_df.rename(columns={"Value": "yield_raw"})

# Convert hg/ha → t/ha if needed
if "hg/ha" in yield_df["Unit"].unique():
    yield_df["yield_t_ha"] = yield_df["yield_raw"] / 10000
else:
    yield_df["yield_t_ha"] = yield_df["yield_raw"]

# Keep only yield column
yield_final = yield_df[["yield_t_ha"]].reset_index(drop=True)

In [172]:
yield_final.head(4)

,yield_t_ha
0,1806.7
1,17912.9
2,6924.5
3,23246.3


In [173]:
yield_final.shape

(2162, 1)

In [221]:
yield_clean["clean_yield_t_ha"] = pd.DataFrame([i for i in yield_final['yield_t_ha'] if i < 300])

print(yield_clean.shape)

(516, 2)


In [222]:
print(yield_clean["clean_yield_t_ha"].describe())

count     72.000000
mean     194.326389
std       76.924201
min       23.000000
25%      139.350000
50%      211.700000
75%      257.650000
max      299.900000
Name: clean_yield_t_ha, dtype: float64


In [223]:
# print(yield_final.describe())
# print(yield_final.value_counts().head(10))

#### Now it's time to concatenat datas together 

In [224]:
all_prov_df = pd.concat([df,Temp_df], axis=1)
all_prov_df = pd.concat([all_prov_df,rainfall_df], axis=1)
all_prov_df = pd.concat([all_prov_df,soil_df], axis=1)
all_prov_df = pd.concat([all_prov_df,yield_clean], axis=1)

In [225]:
all_prov_df

,mean_NDVI,mean_NDWI,Temperature,Rainfall,soil_OC,soil_N,soil_pH,0,clean_yield_t_ha
0,0.034503,-0.120302,24.86,5.46,4.457862,8.649378,9.162534,975.6,255.4
1,0.163364,-0.235962,24.61,5.04,186.690740,142.174011,42.504298,951.0,161.7
2,0.047870,-0.099186,24.56,5.01,186.690740,142.174011,42.504298,950.0,200.0
3,0.113331,-0.148800,24.92,5.03,186.690740,142.174011,42.504298,940.7,259.0
4,0.211293,-0.225793,24.97,4.58,186.690740,142.174011,42.504298,312.8,216.9
...,...,...,...,...,...,...,...,...,...
845,NaN,NaN,NaN,NaN,150.752067,112.491903,59.392828,NaN,NaN
846,NaN,NaN,NaN,NaN,141.564199,112.536212,64.609620,NaN,NaN
847,NaN,NaN,NaN,NaN,120.581660,95.294119,63.023304,NaN,NaN
848,NaN,NaN,NaN,NaN,109.520076,94.725855,66.494269,NaN,NaN


In [226]:
# Let's fill null values with mean of our data
all_prov_df = all_prov_df.fillna(all_prov_df.mean())
all_prov_df.head(5)

,mean_NDVI,mean_NDWI,Temperature,Rainfall,soil_OC,soil_N,soil_pH,0,clean_yield_t_ha
0,0.034503,-0.120302,24.86,5.46,4.457862,8.649378,9.162534,975.6,255.4
1,0.163364,-0.235962,24.61,5.04,186.690740,142.174011,42.504298,951.0,161.7
2,0.047870,-0.099186,24.56,5.01,186.690740,142.174011,42.504298,950.0,200.0
3,0.113331,-0.148800,24.92,5.03,186.690740,142.174011,42.504298,940.7,259.0
4,0.211293,-0.225793,24.97,4.58,186.690740,142.174011,42.504298,312.8,216.9


## Let's apply Random Forest Method to predict yield

In [227]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [228]:
x = all_prov_df.drop(columns=['clean_yield_t_ha', 0])
y = all_prov_df['clean_yield_t_ha'].fillna(all_prov_df['clean_yield_t_ha'].mean())

In [229]:
y

0      255.400000
1      161.700000
2      200.000000
3      259.000000
4      216.900000
          ...    
845    194.326389
846    194.326389
847    194.326389
848    194.326389
849    194.326389
Name: clean_yield_t_ha, Length: 850, dtype: float64

In [230]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [231]:
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(x_train, y_train)

RandomForestRegressor(max_depth=10, random_state=42)

In [247]:
rdm_pred = model.predict(x_test)

In [248]:
print("Predictions:", rdm_pred)
print("MSE:", mean_squared_error(y_test, rdm_pred))

Predictions: [138.93106  197.04218  196.00154  199.76471  196.69945  201.13991
 201.3787   194.32301  191.61325  198.13553  194.32283  194.39183
 191.61783  195.26804  181.70029  194.58904  189.68066  183.20415
 201.66406  190.49977  197.65652  174.87679  219.77704  193.50598
 194.15692  194.3843   194.3587   195.10065  194.82162  197.04657
 195.0372   195.19698  191.96715  189.81703  196.38324  192.23167
 196.11235  183.16006  192.39203  196.77405  195.87152  191.94897
 193.59308  193.68669  198.26749  186.03207  188.55943  194.31828
 190.55551  194.02715  201.91325  207.62352  254.68454  192.8583
 193.24126  194.71478  194.21205  194.36688  194.8213   194.87216
 201.27797  194.43019  194.57716  219.02171  125.124176 192.75786
 223.79868  191.31375  193.60538  203.3185   204.40997  194.9836
 205.44669  204.50218  188.5181   184.22144  194.95795  192.56125
 199.14282  193.31108  204.82205  236.05461  177.53357  199.51466
 194.00742  196.1144   194.57542  194.47333  186.02309  198.96909

#### Let's obtain mean of prediction

In [250]:
rdm_pred_mean = model.predict(x_test)
rdm_pred_mean.mean()

194.95262

## Let's use XGBoost method, to chick which one is more accurate

In [235]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [245]:
model = xgb.XGBRegressor(objective="reg:squarederror")
model.fit(x_train, y_train)

# Predict
xgb_pred = model.predict(x_train)

# print(xgb_pred)

In [246]:
xgb_pred.mean()

195.17642